In [1]:
import tensorflow as tf

from tensorflow.python.keras.models import Sequential, load_model
from tensorflow.python.keras.layers.core import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.layers.convolutional import Conv3D, MaxPooling3D
from tensorflow.python.keras.layers import BatchNormalization
from tensorflow.python.keras.layers import GlobalMaxPool2D
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers import Conv2D
from tensorflow.python.keras.layers import Reshape
from tensorflow.python.keras.optimizers import SGD
from tensorflow.python.keras.utils import np_utils, generic_utils


import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from sklearn import preprocessing

In [5]:
nb_classes = 5

In [6]:
model = Sequential()
#`channels_last` corresponds to inputs with shape `(batch, spatial_dim1, spatial_dim2, spatial_dim3, channels)`
strides = (1,1,1)
kernel_size = (3, 3, 3)
model.add(Conv3D(32, kernel_size, strides=strides, activation='relu', padding='same', input_shape=(32, 64, 96, 3)))
print(model.output_shape)
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(1, 2, 2)))
print(model.output_shape)

model.add(Conv3D(64, kernel_size, strides=strides, activation='relu',padding='same'))
print(model.output_shape)
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(1, 2, 2)))
print(model.output_shape)

model.add(Conv3D(128, kernel_size, strides=strides, activation='relu',padding='same'))
print(model.output_shape)
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(1, 2, 2)))
print(model.output_shape)

model.add(Conv3D(256, kernel_size, strides=strides, activation='relu',padding='same'))
print(model.output_shape)
model.add(BatchNormalization())

model.add(Conv3D(256, kernel_size, strides=strides, activation='relu',padding='same'))
print(model.output_shape)
model.add(BatchNormalization())

model.add(Conv3D(256, kernel_size, strides=strides, activation='relu',padding='same'))
print(model.output_shape)
model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size=(1,8,12)))
print(model.output_shape)

model.add(Reshape((32, 256)))
print(model.output_shape)
model.add(LSTM(256, return_sequences=True))
print(model.output_shape)
model.add(LSTM(256))
print(model.output_shape)

model.add(Dense(256, activation='relu'))
print(model.output_shape)

model.add(Dense(nb_classes, activation='softmax'))
print(model.output_shape)

# model.add(LSTM(256))

(None, 32, 64, 96, 32)
(None, 32, 32, 48, 32)
(None, 32, 32, 48, 64)
(None, 32, 16, 24, 64)
(None, 32, 16, 24, 128)
(None, 32, 8, 12, 128)
(None, 32, 8, 12, 256)
(None, 32, 8, 12, 256)
(None, 32, 8, 12, 256)
(None, 32, 1, 1, 256)
(None, 32, 256)
(None, 32, 256)
(None, 256)
(None, 256)
(None, 5)


In [7]:
PATH = '/home/siddhant/Datasets/20bn Jester/FirstTesting'
print(f'{len(os.listdir(PATH))} classes found')
for clas in os.listdir(PATH):
    print(f'Number of samples in class {clas}: ', end = '')
    print(len(os.listdir(f'{PATH}/{clas}')))


5 classes found
Number of samples in class Doing other things: 8634
Number of samples in class Stop Sign: 3905
Number of samples in class No gesture: 3852
Number of samples in class Thumb Up: 3937
Number of samples in class Swiping Right: 3676


In [8]:
# Resize frames
import matplotlib.image as img

def resize_frame(frame, size = (64,64)):
    frame = img.imread(frame)
    frame = cv2.resize(frame, size)
    return frame

In [9]:
temppath = '/home/siddhant/Datasets/20bn Jester/FirstTesting/Swiping Right/110934'
im = resize_frame(f'{temppath}/001.jpg')
np.shape(im)

(64, 64, 3)

In [10]:
# return gray image
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

In [11]:
def show(img, title="img"):
    cv2.imshow(title, img)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [12]:
import time

In [13]:
frames_new = []
classes = os.listdir(PATH)
print(classes)
frames = os.listdir(f"{PATH}/Stop Sign/31/")
frames.sort()

st = time.time()
for frame in frames:
    frames_new.append(resize_frame(f'{PATH}/Stop Sign/31/{frame}', size = (96, 64)))
e1 = time.time()
print(f"Time of exectution: {e1 - st}")

for frame in frames:
    frames_new.append(cv2.imread(f'{PATH}/Stop Sign/31/{frame}'))
e2 = time.time()
print(f"Time of exectution: {e2 - e1}")

print(len(frames_new))

['Doing other things', 'Stop Sign', 'No gesture', 'Thumb Up', 'Swiping Right']
Time of exectution: 0.2396857738494873
Time of exectution: 0.09943580627441406
64


In [14]:
import random
import os

def picker(sample_count):
    L = []
    for cl in sample_count:
        L = L + [(f'{cl[1]}')]*cl[0]
    random.shuffle(L)
    return L

def pick_sample(shuffled_list):
    a = random.choice(shuffled_list)
    return a

def VideoDataGenerator(data_path, frame_dim, batch_size = 8):
    '''The data_path must contain folders containing samples from only that class.
    The samples should be homogenously numerated. (eg - 001.jpg, 002.jpg etc) for each frame of the video. 
    Enter frame_dim as tuple of dimensions in the format: (x-axis, y-axis)
    '''
    def resize_frame(frame, size = frame_dim):
        frame = img.imread(frame)
        frame = cv2.resize(frame, size)
        return frame
    if data_path[-1] != '/':
        data_path = f"{data_path}/"
    
    classes = os.listdir(data_path)
    print(f"{len(classes)} classes found in folder")
    
    class_target_map = [[classes[i], i] for i in range(len(classes))]
    samples_count = [[len(os.listdir(f"{data_path}{classes[i]}")), i] for i in range(len(classes))]
    picker_list = picker(samples_count)
    
    sample_names = [[] for i in range(len(classes))]
    for i in range(len(classes)):
        sample_names[i] = os.listdir(f'{data_path}{classes[i]}')
    
    while True:
        samples_pushed = []
        offset = 0
        total_samples = 0
        c = 0
        for cl in sample_names:
            if not cl:
                c += 1
            if c == len(classes) - 1:
                break
        X = []
        y = []
        for cl in classes:
            total_samples = total_samples + len(os.listdir(f'{data_path}{cl}'))
            
        for batch_iter in range(batch_size):
            sample_class = pick_sample(picker_list)
            picker_list.remove(sample_class)
            sample_class = int(sample_class)
            if not sample_names[sample_class]: 
                batch_iter -= 1
                continue
            vid_sample = f'{data_path}{classes[sample_class]}/{sample_names[sample_class][0]}'
            #fuck up hoga toh ye^ line me hoga
            frames = os.listdir(vid_sample)
            frames.sort()
            vid = []
            for frame in frames:
                frame = resize_frame(f'{vid_sample}/{frame}')
                try:
                    #frame = resize_frame(frame)
                    vid.append(frame)
                except:
                    print(f'Sample with tag - {sample_names[sample_class][0]} is broken and skipped.')
                    continue
            #print(np.shape(vid))
            #vid = np.reshape(vid, (96, 64, 32, 3))
            X.append(vid)
            y.append(sample_class)
            
            sample_names[sample_class].pop(0)
        X = np.array(X)
        y = np.array(y)
        yield X,y

In [15]:
training_path = '/home/siddhant/Datasets/20bn Jester/FirstTesting'
testing_path = '/home/siddhant/Datasets/20bn Jester/FirstTestingTEST'
validation_path = '/home/siddhant/Datasets/20bn Jester/FirstTestingVAL'

batchsize = 2
training_set = VideoDataGenerator(training_path, frame_dim=(96,64), batch_size=batchsize)
testing_set = VideoDataGenerator(testing_path, frame_dim=(96,64), batch_size=batchsize)
validation_set = VideoDataGenerator(validation_path, frame_dim=(96,64), batch_size=batchsize)

In [16]:
sgd = tf.keras.optimizers.SGD(lr=0.001)
model.compile(optimizer=sgd, loss= 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [17]:
inst = next(training_set)

5 classes found in folder


In [18]:
np.shape(inst[0])

(2, 32, 64, 96, 3)

In [19]:
sgd = tf.keras.optimizers.SGD(lr=0.001)
model.compile(optimizer=sgd, loss= 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [20]:
import sys
sys.getsizeof(inst[0])

1179808

In [32]:
model.load_weights('main.h5')

In [33]:
checkpoints = tf.keras.callbacks.ModelCheckpoint(
    os.getcwd(),
    monitor="val_loss",
    verbose=0,
    save_best_only=False,
    save_weights_only=False,
    mode="auto",
    save_freq="epoch"
)


In [34]:
training_history = model.fit_generator(training_set, use_multiprocessing=True,
                                       epochs =20,validation_data = validation_set , verbose = 1 , validation_steps = 1,
                                       steps_per_epoch=24000//4, callbacks = [checkpoints])

Epoch 1/20
 123/6000 [..............................] - ETA: 1:03:14 - loss: 0.1263 - accuracy: 0.9512

Process Keras_worker_ForkPoolWorker-10:
Traceback (most recent call last):
  File "/home/siddhant/anaconda3/envs/tf_gpu/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/siddhant/anaconda3/envs/tf_gpu/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/siddhant/anaconda3/envs/tf_gpu/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/siddhant/anaconda3/envs/tf_gpu/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/home/siddhant/anaconda3/envs/tf_gpu/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/siddhant/anaconda3/envs/tf_gpu/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/siddhant/anaconda3/envs/tf_gpu/lib/python3.7/multiprocessing/connection.py", li

KeyboardInterrupt: 

KeyboardInterrupt


Epoch 1/20


In [31]:
model.save_weights(filepath='main.h5')